In [1]:
# %load ../config/defaults.py
import os
import sys
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs

In [98]:
# Setup notebook
nbconfig = Nb.setup_notebook()

Please check output/fbgn2chrom.tsv. If it does not exist, run bin/fbgn2chrom.py
last updated: 2018-10-12 
Git hash: da3bbc4919814182d8c6894679de675c7d0adc44


In [99]:
# Connect to data store
store = pd.HDFStore('../sra.h5', mode='r')

In [147]:
libstrat = pd.read_parquet('../output/metadata-wf/select_library_strategy.parquet')

In [136]:
libstrat[libstrat.str.contains('GRO')]

srx
SRX026307     GRO-Seq|ChIP-Seq|RNA-Seq
SRX1011104                     GRO-Seq
SRX1011106                     GRO-Seq
SRX1011107                     GRO-Seq
SRX1011108                     GRO-Seq
SRX1011109                     GRO-Seq
SRX1033610                     GRO-Seq
SRX1033611                     GRO-Seq
SRX1033612                     GRO-Seq
SRX1033613                     GRO-Seq
SRX1120707                     GRO-Seq
SRX1120708                     GRO-Seq
SRX1120709                     GRO-Seq
SRX1120710                     GRO-Seq
SRX1120711                     GRO-Seq
SRX1120712                     GRO-Seq
SRX1120713                     GRO-Seq
SRX1120714                     GRO-Seq
SRX1120715                     GRO-Seq
SRX1120716                     GRO-Seq
SRX215615                      GRO-Seq
SRX215616                      GRO-Seq
SRX215617                      GRO-Seq
SRX215618                      GRO-Seq
SRX643398                      GRO-Seq
SRX643399            

In [137]:
libstrat[libstrat.str.contains('PRO')]

srx
SRX1562306    PRO-Seq
SRX1562307    PRO-Seq
SRX1562308    PRO-Seq
SRX1562309    PRO-Seq
SRX1562310    PRO-Seq
SRX1562311    PRO-Seq
SRX1562312    PRO-Seq
SRX1562313    PRO-Seq
SRX1562314    PRO-Seq
SRX1562315    PRO-Seq
SRX1562316    PRO-Seq
SRX1562317    PRO-Seq
SRX1775533    PRO-Seq
SRX1775534    PRO-Seq
SRX1997358    PRO-Seq
SRX1997359    PRO-Seq
SRX1997360    PRO-Seq
SRX1997361    PRO-Seq
SRX203291     PRO-Seq
SRX203292     PRO-Seq
SRX203294     PRO-Seq
SRX203295     PRO-Seq
SRX203296     PRO-Seq
Name: Fear_et_al_library_strategy, dtype: object

In [145]:
libstrat[libstrat.str.contains('4C')]

srx
ERX1432621    4C-Seq|ChIP-Seq|WGS
ERX1495767                 4C-Seq
ERX364701                  4C-Seq
ERX364702                  4C-Seq
ERX364703                  4C-Seq
ERX364704                  4C-Seq
ERX364705                  4C-Seq
ERX364706                  4C-Seq
ERX364707                  4C-Seq
ERX364708                  4C-Seq
ERX364709                  4C-Seq
ERX364710                  4C-Seq
ERX364711                  4C-Seq
ERX364712                  4C-Seq
ERX364713                  4C-Seq
ERX364714                  4C-Seq
ERX364715                  4C-Seq
ERX364716                  4C-Seq
ERX364717                  4C-Seq
ERX364718                  4C-Seq
ERX364719                  4C-Seq
ERX364720                  4C-Seq
ERX364721                  4C-Seq
ERX364722                  4C-Seq
ERX364723                  4C-Seq
ERX364724                  4C-Seq
ERX364725                  4C-Seq
ERX364726                  4C-Seq
ERX364727                  4C-Seq
ERX364728 

In [148]:
libstrat.to_csv('~/library_strategy.tsv', sep='\t')